In [ ]:
#DUAL MOMENTUM QUARTERLY ROTATION
#Developers: Kory Hoang & Mohammed Khalfan
#For collaboration requests, please email: hoang.kory@gmail.com
import datetime
from quantopian.research import run_pipeline, prices
from quantopian.research.experimental import history
import numpy as np
import pandas as pd

currently_holding = []
#schedule_function(clear, date_rules.month_end(), time_rules.market_open(hours=0,minutes=1))
#schedule_function(rebalance, date_rules.month_end(), time_rules.market_close(hours=6,minutes=0))
#schedule_function(record_vars, date_rules.every_day(), time_rules.market_close())
#set_benchmark(symbol('SPY'))

start_date = '2019-8-28'
end_date = '2019-9-30'
today = datetime.datetime(2019,9,30)
replace_prices ={'QQQ':75.95, 'IJR':21.32, 'MDY':84.67, 'EEM':46.18, 'EFA':35.39, 'EDV':226.22, 'GLD':1512.76, 'AGG':30.49, 'TLT':323.97, 'USO':164.86 }

is_run_history = True

risk = 1.5
buy_count = 4
mom_period = 22

#assets_normal = symbols('QQQ','IJR','MDY','EEM','EFA','EDV','GLD','USO')
#assets_12 = symbols('QQQ','IJR','MDY','EEM','EFA','EDV','GLD')
assets = [symbols('QQQ'),symbols('IJR'),symbols('MDY'),symbols('EEM'),symbols('EFA'),symbols('EDV'),symbols('GLD')]
#assets = symbols('SVXY','TQQQ','QQQ','IJR','MDY','EEM','EFA','TMF','EDV','GLD','UCO','USO')
#assets = 
#assets = symbols('EEM','EFA','EDV','GLD','USO','XLB','XLY','XLE','XLF','XLV','XLI','IYR','XLK','IYZ','XLU')
safe = symbols('AGG')
tlt = symbols('TLT')
uso = symbols('USO')

positions = {}# format { 'MTCH':[411,0], 'IAC':[131,0], 'MKTX':[91,0] } #liang
capital = 100000

trades = []

if is_run_history:
    hist_prices = history(symbols(replace_prices.keys()),'price',end_date,end_date,'daily')
    for col in hist_prices.columns:
        if col.symbol in replace_prices:
            replace_prices[col.symbol] = hist_prices[col][-1] 

for pos in positions.keys():
    if pos in replace_prices:
        positions[pos][1] = replace_prices[pos]
        
def trade_target_percent(security, percent):
    if security not in positions:
        positions[security] = [0,0]
        positions[security][1] = replace_prices[security.symbol]
        
    quantity = capital * percent / replace_prices[security.symbol]
    trade_quantity = quantity - positions[security][0]
    positions[security][0] = quantity
    
    if trade_quantity != 0:
        trades.append((security, trade_quantity, replace_prices[security.symbol], trade_quantity * replace_prices[security.symbol]))

#def clear(context, data):

#    today = get_datetime()
#    if not (today.month == 12 or today.month == 3 or today.month == 6 or today.month == 9):
#        return
    
#    print("1" + str(context.currently_holding))
#    for s in context.currently_holding:
#        order_target_percent(s, 0)
#    context.currently_holding = []   

#def rebalance(context, data):
if (today.month == 12 or today.month == 3 or today.month == 6 or today.month == 9):
       
    hist = history(assets,'price',start_date,end_date,'daily')

    for col in hist.columns:
        if col.symbol in replace_prices:
            hist[col][-1] = replace_prices[col.symbol]
        
    #hist = data.history(, 'price', context.mom_period + 1, '1d')  
    mom = (hist.iloc[-1]/hist.iloc[0]) - 1.0  
    mom = mom.dropna()  
    top_assets = mom.sort_values().index[-4:]
    top_assets_values = mom.sort_values()[-4:]
    
    safe_percent = 0
    tlt_percent = 0
    
    for x in range(len(top_assets)):
        if top_assets_values[x] > 0:
            if top_assets[x].symbol in replace_prices:  
                trade_target_percent(top_assets[x], risk/buy_count)
                currently_holding.append(top_assets[x])
        else:
            #hist = data.history(context.safe, 'price', context.mom_period + 1, '1d') 
            hist = history([safe],'price',start_date,end_date,'daily')
            for col in hist.columns:
                if col.symbol in replace_prices:
                    hist[col][-1] = replace_prices[col.symbol]
            
            mom = (hist[safe][-1]/hist[safe][0]) - 1.0
            if mom > 0:
                safe_percent += risk / buy_count
                if safe not in currently_holding: currently_holding.append(safe)
            else:
                tlt_percent += risk / buy_count
                if tlt not in currently_holding: currently_holding.append(tlt)

    top_up = 4 - len(top_assets)
    if top_up > 0:
        for x in range(top_up):
            hist = history([safe],'price',start_date,end_date,'daily')
            for col in hist.columns:
                if col.symbol in replace_prices:
                    hist[col][-1] = replace_prices[col.symbol]

            mom = (hist[safe][-1]/hist[safe][0]) - 1.0
            if mom > 0:
                safe_percent += risk / buy_count
                if safe not in currently_holding: currently_holding.append(safe)
            else:
                tlt_percent += risk / buy_count
                if tlt not in currently_holding: currently_holding.append(tlt)
                        
    print(safe_percent, tlt_percent)
    if safe_percent > 0.0 and safe in replace_prices:  
        trade_target_percent(safe, safe_percent)                        
                        
    if tlt_percent > 0.0 and tlt in replace_prices:  
        trade_target_percent(tlt, tlt_percent)
        
    for position in positions:
        if position not in currently_holding:
            trade_target_percent(position, 0)
            
#    print("2" + str(context.currently_holding)  )

    
trades    

In [ ]:
positions

In [ ]:
hist_prices

In [ ]:
replace_prices

In [ ]:
hist